Для своевременного принятия решения на основе данных, минимизации ошибок и быстрого достижения целей, необходимо построить аналитический дашборд. Интерактивная информационная панель должна наглядно представлять, визуализировать и анализировать данные. Для этого необходимо подключить к системе источник данных, т.е. подключиться к базе данных, настроить регламент обновления и уровни доступа.
Аналитическая система должна позволить пользователю:
–	Автоматизировать работу с данными;
–	Анализировать все в реальном времени;
–	Видеть все главные метрики на одном экране;
–	Легко считывать информацию и делать выводы.


In [ ]:

import sqlite3
import pandas as pd

# Подключение к базе данных
conn = sqlite3.connect('metro_data.db')

# Загрузка данных
query = '''
SELECT
    t1.станция AS Станция,
    t1.линия AS Линия,
    t1.номер_билета AS Номер_билета,
    t1.время_входа AS Время_входа,
    t1.время_выхода AS Время_выхода,
    (julianday(t1.время_выхода) - julianday(t1.время_входа)) * 1440 AS Продолжительность_поездки,
    t2.среднее_время_ожидания AS Среднее_время_ожидания,
    t2.уровень_комфорта AS Уровень_комфорта
FROM поездки AS t1
JOIN станция_инфо AS t2 ON t1.станция = t2.станция
'''

df = pd.read_sql_query(query, conn)

# Закрытие соединения с базой данных
conn.close()

print(df.head())


Создание интерактивного дашборда с Plotly Dash

In [ ]:
import dash
from dash import dcc, html
import dash_table
import plotly.express as px
from dash.dependencies import Input, Output

# Инициализация Dash-приложения
app = dash.Dash(__name__)

# Расчёт метрик
total_load = df['Станция'].value_counts().reset_index()
total_load.columns = ['Станция', 'Количество пассажиров']
average_load = df['Станция'].value_counts().mean()
df['Пропускная способность, %'] = df['Станция'].map(df['Станция'].value_counts() / average_load * 100)

# Визуализации
fig_load = px.bar(total_load, x='Станция', y='Количество пассажиров', title='Общая загруженность станций')
fig_duration = px.histogram(df, x='Продолжительность_поездки', title='Распределение продолжительности поездок')

# Макет Dash-приложения
app.layout = html.Div([
    html.H1("Аналитический дашборд метрополитена"),

    html.Div([
        dcc.Graph(figure=fig_load),
        dcc.Graph(figure=fig_duration),
    ], style={'display': 'flex', 'flex-direction': 'row'}),

    html.Div([
        dash_table.DataTable(
            id='table',
            columns=[{"name": i, "id": i} for i in df.columns],
            data=df.to_dict('records'),
            page_size=10,
        ),
    ])
])

if __name__ == '__main__':
    app.run_server(debug=True)


Регламент обновления данных
Для обновления данных можно использовать встроенные возможности Dash для периодического обновления данных. Например, с использованием dcc.Interval для автоматического обновления данных каждые несколько минут.

Уровни доступа
Для управления доступом можно использовать такие инструменты как Flask-Login для создания авторизации и аутентификации пользователей.